## Import modules

In [1]:
import sys
import time
import json
import pickle

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot
import concurrent.futures

from XRootD import client
from XRootD.client.flags import DirListFlags, StatInfoFlags, OpenFlags, MkDirFlags, QueryCode

Import local classes from utils

In [2]:
%load_ext jupyternotify
%autonotify -a 10

%load_ext autoreload
%autoreload 1
%aimport utils.ObjectExtractor
%aimport utils.PlotMaker
%aimport utils.HistogramContainer
%aimport utils.HistogramCalculator
OE = utils.ObjectExtractor
PM = utils.PlotMaker
HCont = utils.HistogramContainer
HCalc = utils.HistogramCalculator

<IPython.core.display.Javascript object>

## Global initialization

In [3]:
print(sys.version_info)
# num_cores = multiprocessing.cpu_count()
# print(num_cores)


executor = concurrent.futures.ThreadPoolExecutor(48)

plt.style.use('default')
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams.update({'font.size': 10})

numCuts = np.arange(0,16)

branch_path = 'SREffi_gbm'

labels = [ f'cut{cut}' for cut in numCuts ]
cut_descriptions = [
    'cut1: MET/MHT trigger fired (120 GeV)',
    'cut2: j1 pT > 120 GeV, <= 2j w/ pT > 30 GeV',
    'cut3: mu1 pT > 5 GeV, 0.1 < |dxy| < 700 cm',
    'cut4: mu2 pT > 5 GeV, 0.1 < |dxy| < 700 cm',
    r'cut5: $|\Delta\Phi$(MET, mu pair)| < 0.4',
    'cut6: ',
    'cut7: ',
    'cut8: ',
    'cut9: ',
    'cut10:',
    'cut11:',
    'cut12:',
    'cut13:',
    'cut14:',
    'cut15:',
]

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [4]:
all_plot_vars = ['metpt', 'jetpt','metjetphi', 'metmuphi', 'leadingmupt', 'subleadingmupt','recodr', 'recovertex']
#all_plot_vars = ['reco_PF_MET_pt', 'reco_PF_jet_pt','metjetphi', 'metmuphi', 'leadingmupt', 'subleadingmupt', 'recodr', 'recovertex']
plot_vars_metjet = all_plot_vars[0:4] #['metpt', 'jetpt', 'metjetphi', 'metmuphi']
plot_vars_muons = all_plot_vars[4:8] #['leadingmupt', 'subleadingmupt', 'recodr', 'recovertex']
cutflow_vars = ['cutflow_incl', 'cutflow_excl']
all_plot_xlabels = [
    'MET [GeV]', 'Leading jet pT [GeV]', '$\Delta\Phi$(MET, jet)', '$\Delta\Phi$(MET, di-muon)',
    'Leading muon pT [GeV]', 'Subleading muon pT [GeV]', 'dR(muons)', 'Di-muon vertex [cm]']

In [5]:
histos = {}
all_bins = {}
for plot_var in all_plot_vars:
    histos[plot_var] = {}
    all_bins[plot_var] = 60
histos['cutflow_incl'] = {}
histos['cutflow_excl'] = {}
histos['sumgenwgt'] = {}

## Process signal

In [6]:
masses = [('60p0','20p0'),('6p0','2p0'),('52p5','5p0'),('5p25','0p5')]
def print_masses(mass):
    return f'({float(mass[0].replace("p","."))-float(mass[1].replace("p","."))/2}, ' + \
           f'{float(mass[0].replace("p","."))+float(mass[1].replace("p","."))/2}) GeV'
    
mchis = dict([(mass[0], print_masses(mass)) for mass in masses])
ctaus = [10]#, 10, 100, 1000]

labels = [ f'cut{cut}' for cut in numCuts ]
cut_descriptions = [
    'cut1: MET/MHT trigger fired (120 GeV)',
    'cut2: j1 pT > 120 GeV, <= 2j w/ pT > 30 GeV',
    'cut3: mu1 pT > 5 GeV, 0.1 < |dxy| < 700 cm',
    'cut4: mu2 pT > 5 GeV, 0.1 < |dxy| < 700 cm',
    r'cut5: $|\Delta\Phi$(MET, mu pair)| < 0.4',
]


base_dir = '../iDM-analysis-skimming/washAOD/SROptimization/'
def filename(Mchi, dMchi, ctau): 
    return base_dir + f'Mchi-{Mchi}_dMchi-{dMchi}_ctau-{ctau}.root'

In [7]:
trees_gbm = dict()
gen_info_gbm = dict()

In [8]:
## new signal input
with open('config/sig.json') as sigs_json_file:
    sigs = json.load(sigs_json_file)

xrdfs = client.FileSystem("root://cmseos.fnal.gov/")

redirector = 'root://cmsxrootd.fnal.gov'
sig_base_dir = '/store/group/lpcmetx/iDM/Ntuples/2018/signal/'
files = {}

for sig, properties in sigs.items():
    files[sig] = []
    status, listing = xrdfs.dirlist(f'{sig_base_dir}/{properties["dir"]}', DirListFlags.STAT)
    for file in listing:
        if '.root' in file.name:
            files[sig].append(f'{redirector}/{sig_base_dir}/{properties["dir"]}/{file.name}')
num_files_total = np.sum(np.array([len(files[i]) for i in files]))
print(num_files_total)
[(i, len(files[i])) for i in files]

1 of 4: ../iDM-analysis-skimming/washAOD/SROptimization/Mchi-60p0_dMchi-20p0_ctau-10.root
2 of 4: ../iDM-analysis-skimming/washAOD/SROptimization/Mchi-6p0_dMchi-2p0_ctau-10.root
3 of 4: ../iDM-analysis-skimming/washAOD/SROptimization/Mchi-52p5_dMchi-5p0_ctau-10.root
4 of 4: ../iDM-analysis-skimming/washAOD/SROptimization/Mchi-5p25_dMchi-0p5_ctau-10.root


[('Mchi-6p0_dMchi-2p0_ctau-1', 1),
 ('Mchi-6p0_dMchi-2p0_ctau-10', 1),
 ('Mchi-6p0_dMchi-2p0_ctau-100', 1),
 ('Mchi-6p0_dMchi-2p0_ctau-1000', 1),
 ('Mchi-60p0_dMchi-20p0_ctau-1', 1),
 ('Mchi-60p0_dMchi-20p0_ctau-10', 1),
 ('Mchi-60p0_dMchi-20p0_ctau-100', 1),
 ('Mchi-60p0_dMchi-20p0_ctau-1000', 1),
 ('Mchi-52p5_dMchi-5p0_ctau-1', 1),
 ('Mchi-52p5_dMchi-5p0_ctau-10', 1),
 ('Mchi-52p5_dMchi-5p0_ctau-100', 1),
 ('Mchi-52p5_dMchi-5p0_ctau-1000', 1),
 ('Mchi-5p25_dMchi-0p5_ctau-1', 1),
 ('Mchi-5p25_dMchi-0p5_ctau-10', 1),
 ('Mchi-5p25_dMchi-0p5_ctau-100', 1),
 ('Mchi-5p25_dMchi-0p5_ctau-1000', 1)]

In [9]:
%%time

MAX_FILES=None # To load all possible files
# MAX_FILES=1 # For testing

### Initialize empty dicts of histograms 
# histos = {}
# all_bins = {}
# for plot_var in all_plot_vars:
#     histos[plot_var] = {}
#     all_bins[plot_var] = 60
# histos['cutflow_incl'] = {}
# histos['cutflow_excl'] = {}
# histos['sumgenwgt'] = {}

global_file_counter = 1

for sig in sigs:
    
    print(f'Processing signal {sig} ({(list(sigs.keys())).index(sig)+1}/{len(sigs)})')
    
    ### Initialize histograms as empty HistogramContainers
    for plot_var in all_plot_vars:
        histos[plot_var][sig] = HCont.HistogramContainer(all_bins[plot_var])
    histos['cutflow_incl'][sig] = np.zeros(len(numCuts))
    histos['cutflow_excl'][sig] = np.zeros(len(numCuts))
    histos['sumgenwgt'][sig] = 0.0
    
    ### Load data
    file_counter = 1
    for file in files[sig][slice(0,MAX_FILES)]:
        
        if file_counter % 10 == 1:
            print(f'Reading file {file_counter} of {len(files[sig])},'
                  f' global {global_file_counter} of {num_files_total}'
                  f' ({100*(global_file_counter-1)/num_files_total:.2f}%)')
            with open('histos_temp.dat', 'wb') as histos_file:
                pickle.dump(histos, histos_file)
        file_counter += 1
        global_file_counter += 1
        
        ### Open ROOT file and get tree
        tree = uproot.open(file)[branch_path + '/reco']
        
        ### Make pandas dataframes and create all objects that will be passed to histo functions
        obj_extractor = OE.ObjectExtractor(tree)
        objects = obj_extractor.get_all()
            
        ## Add to sum of genwgts
        histos['sumgenwgt'][sig] += np.sum(objects['gen_wgt'])
        
        ### Calculate histograms and cutflows
        histo_maker = HCalc.HistogramCalculator(objects, sig)
            
        ### Cutflows
        incl, excl = histo_maker.cutflows()
        histos['cutflow_incl'][sig] += incl
        histos['cutflow_excl'][sig] += excl
        
        ### Histograms
        for plot_var in all_plot_vars:
            new_hist = eval(f'histo_maker.{plot_var}()')
            histos[plot_var][sig] += new_hist


Processing signal Mchi-6p0_dMchi-2p0_ctau-1 (1/16)
Reading file 1 of 1, global 1 of 16 (0.00%)
Sample "" does not have either pileup or weight information
Processing signal Mchi-6p0_dMchi-2p0_ctau-10 (2/16)
Reading file 1 of 1, global 2 of 16 (6.25%)
Sample "" does not have either pileup or weight information
Processing signal Mchi-6p0_dMchi-2p0_ctau-100 (3/16)
Reading file 1 of 1, global 3 of 16 (12.50%)
Sample "" does not have either pileup or weight information
Processing signal Mchi-6p0_dMchi-2p0_ctau-1000 (4/16)
Reading file 1 of 1, global 4 of 16 (18.75%)
Sample "" does not have either pileup or weight information
Processing signal Mchi-60p0_dMchi-20p0_ctau-1 (5/16)
Reading file 1 of 1, global 5 of 16 (25.00%)
Sample "" does not have either pileup or weight information
Processing signal Mchi-60p0_dMchi-20p0_ctau-10 (6/16)
Reading file 1 of 1, global 6 of 16 (31.25%)
Sample "" does not have either pileup or weight information
Processing signal Mchi-60p0_dMchi-20p0_ctau-100 (7/16)


<IPython.core.display.Javascript object>

In [8]:
luminosity = 59.97 * 1000 # 1/pb
for sig, properties in sigs.items():
    properties['weight'] = luminosity * properties['xsec'] / histos['sumgenwgt'][sig]
#     except KeyError:
#         properties['weight'] = 1
for sig, properties in sigs.items():
    try:
        print(sig, luminosity * properties['xsec'] / histos['sumgenwgt'][sig], histos['sumgenwgt'][sig])
    except KeyError: pass
    
with open('histos_signal_objects_gbm.dat', 'wb') as histos_file:
    pickle.dump(histos, histos_file)

Mchi-6p0_dMchi-2p0_ctau-1 7053.7400881057265 45400.0
Mchi-6p0_dMchi-2p0_ctau-10 7057.159857199524 45378.0
Mchi-6p0_dMchi-2p0_ctau-100 7053.7400881057265 45400.0
Mchi-6p0_dMchi-2p0_ctau-1000 7053.7400881057265 45400.0
Mchi-60p0_dMchi-20p0_ctau-1 2279.205722216291 140505.0
Mchi-60p0_dMchi-20p0_ctau-10 2278.4759871931697 140550.0
Mchi-60p0_dMchi-20p0_ctau-100 2278.897554866073 140524.0
Mchi-60p0_dMchi-20p0_ctau-1000 2276.5646771120655 140668.0
Mchi-52p5_dMchi-5p0_ctau-1 2031.5401499676466 157634.0
Mchi-52p5_dMchi-5p0_ctau-10 2033.2556618688136 157501.0
Mchi-52p5_dMchi-5p0_ctau-100 2040.497763504989 156942.0
Mchi-52p5_dMchi-5p0_ctau-1000 2036.4363613239643 157255.0
Mchi-5p25_dMchi-0p5_ctau-1 5487.034593835136 58363.0
Mchi-5p25_dMchi-0p5_ctau-10 5491.174402853272 58319.0
Mchi-5p25_dMchi-0p5_ctau-100 5485.436793422405 58380.0
Mchi-5p25_dMchi-0p5_ctau-1000 5485.436793422405 58380.0


## Process backgrounds

In [10]:
with open('config/bkgs2.json') as bkgs_json_file:
    bkgs = json.load(bkgs_json_file)

In [11]:
xrdfs = client.FileSystem("root://cmseos.fnal.gov/")

redirector = 'root://cmsxrootd.fnal.gov'
bkg_base_dir = '/store/group/lpcmetx/iDM/Ntuples/2018/backgrounds'
files = {}

for bkg, properties in bkgs.items():
    files[bkg] = []
    status, listing = xrdfs.dirlist(f'{bkg_base_dir}/{properties["dir"]}', DirListFlags.STAT)
    for file in listing:
        if '.root' in file.name:
            files[bkg].append(f'{redirector}/{bkg_base_dir}/{properties["dir"]}/{file.name}')

In [12]:
num_files_total = np.sum(np.array([len(files[i]) for i in files]))
print(num_files_total)
[(i, len(files[i])) for i in files]

1364


[('DYJetsToLL', 490),
 ('QCD_bEnriched_HT100to200', 82),
 ('QCD_bEnriched_HT200to300', 63),
 ('QCD_bEnriched_HT300to500', 13),
 ('QCD_bEnriched_HT500to700', 24),
 ('QCD_bEnriched_HT700to1000', 11),
 ('QCD_bEnriched_HT1000to1500', 1),
 ('QCD_bEnriched_HT1500to2000', 1),
 ('QCD_bEnriched_HT2000toINF', 1),
 ('TTTo2L2Nu', 172),
 ('TTJets', 30),
 ('TT_diLept', 5),
 ('WJetsToLNu_HT-70To100', 168),
 ('WJetsToLNu_HT-100To200', 66),
 ('WJetsToLNu_HT-200To400', 37),
 ('WJetsToLNu_HT-400To600', 7),
 ('ZJetsToNuNu_HT-100To200', 54),
 ('ZJetsToNuNu_HT-200To400', 41),
 ('ZJetsToNuNu_HT-400To600', 36),
 ('ZJetsToNuNu_HT-600To800', 12),
 ('ZJetsToNuNu_HT-800To1200', 10),
 ('ZJetsToNuNu_HT-1200To2500', 1),
 ('ZJetsToNuNu_HT-2500ToInf', 1),
 ('WWJJToLNuLNu', 1),
 ('WWTo2L2Nu', 20),
 ('WZTo3LNu', 1),
 ('ZZTo2L2Nu', 16)]

In [22]:
%%time

# MAX_FILES=None # To load all possible files
MAX_FILES=2 # For testing

### Initialize empty dicts of histograms 
# histos = {}
# all_bins = {}
# for plot_var in all_plot_vars:
#     histos[plot_var] = {}
#     all_bins[plot_var] = 60
# histos['cutflow_incl'] = {}
# histos['cutflow_excl'] = {}
# histos['sumgenwgt'] = {}

global_file_counter = 1

for bkg in bkgs:
    
    print(f'Processing background {bkg} ({(list(bkgs.keys())).index(bkg)+1}/{len(bkgs)})')
    
    ### Initialize histograms as empty HistogramContainers
    for plot_var in all_plot_vars:
        histos[plot_var][bkg] = HCont.HistogramContainer(all_bins[plot_var])
    histos['cutflow_incl'][bkg] = np.zeros(len(numCuts))
    histos['cutflow_excl'][bkg] = np.zeros(len(numCuts))
    histos['sumgenwgt'][bkg] = 0.0
    
    ### Load data
    file_counter = 1
    for file in files[bkg][slice(0,MAX_FILES)]:
        
        if file_counter % 10 == 1:
            print(f'Reading file {file_counter} of {len(files[bkg])},'
                  f' global {global_file_counter} of {num_files_total}'
                  f' ({100*(global_file_counter-1)/num_files_total:.2f}%)')
            with open('histos_temp.dat', 'wb') as histos_file:
                pickle.dump(histos, histos_file)
        file_counter += 1
        global_file_counter += 1
        
        ### Open ROOT file and get tree
        tree = uproot.open(file)[branch_path + '/cutsTree']
        
        ### Make pandas dataframes and create all objects that will be passed to histo functions
        obj_extractor = OE.ObjectExtractor(tree)
        objects = obj_extractor.get_all()
            
        ## Add to sum of genwgts
        histos['sumgenwgt'][bkg] += np.sum(objects['genwgt'])
        
        ### Calculate histograms and cutflows
        histo_maker = HCalc.HistogramCalculator(objects, bkg)
            
        ### Cutflows
        incl, excl = histo_maker.cutflows()
        histos['cutflow_incl'][bkg] += incl
        histos['cutflow_excl'][bkg] += excl
        
        ### Histograms
        for plot_var in all_plot_vars:
            new_hist = eval(f'histo_maker.{plot_var}()')
            histos[plot_var][bkg] += new_hist

Processing background DYJetsToLL (1/27)
Reading file 1 of 490, global 1 of 1364 (0.00%)
Processing background QCD_bEnriched_HT100to200 (2/27)
Reading file 1 of 82, global 3 of 1364 (0.15%)
Processing background QCD_bEnriched_HT200to300 (3/27)
Reading file 1 of 63, global 5 of 1364 (0.29%)
Processing background QCD_bEnriched_HT300to500 (4/27)
Reading file 1 of 13, global 7 of 1364 (0.44%)
Processing background QCD_bEnriched_HT500to700 (5/27)
Reading file 1 of 24, global 9 of 1364 (0.59%)
Processing background QCD_bEnriched_HT700to1000 (6/27)
Reading file 1 of 11, global 11 of 1364 (0.73%)
Processing background QCD_bEnriched_HT1000to1500 (7/27)
Reading file 1 of 1, global 13 of 1364 (0.88%)
Processing background QCD_bEnriched_HT1500to2000 (8/27)
Reading file 1 of 1, global 14 of 1364 (0.95%)
Processing background QCD_bEnriched_HT2000toINF (9/27)
Reading file 1 of 1, global 15 of 1364 (1.03%)
Processing background TTTo2L2Nu (10/27)
Reading file 1 of 172, global 16 of 1364 (1.10%)
Processi

<IPython.core.display.Javascript object>

In [23]:
with open('histos_temp.dat', 'wb') as histos_file:
    pickle.dump(histos, histos_file)

In [14]:
luminosity = 59.97 * 1000 # 1/pb
for bkg, properties in bkgs.items():
    properties['weight'] = luminosity * properties['xsec'] / histos['sumgenwgt'][bkg]
#     except KeyError:
#         properties['weight'] = 1

In [16]:
for bkg, properties in bkgs.items():
    try:
        print(bkg, luminosity * properties['xsec'] / histos['sumgenwgt'][bkg], histos['sumgenwgt'][bkg])
    except KeyError: pass

DYJetsToLL 1532.446448558506 208972.9140625
QCD_bEnriched_HT100to200 121663.67032035017 553052.03125
QCD_bEnriched_HT200to300 12747.545378030525 377154.5625
QCD_bEnriched_HT300to500 939.2816048504089 1065600.875
QCD_bEnriched_HT500to700 234.88200260330424 377873.140625
QCD_bEnriched_HT700to1000 72.78128544965914 244473.546875
QCD_bEnriched_HT1000to1500 5.668163836746107 491976.78125
QCD_bEnriched_HT1500to2000 0.5451332217184982 409126.46875
QCD_bEnriched_HT2000toINF 0.0979757226515774 396573.375
TTTo2L2Nu 1.8448089475505214 22335856.0
TTJets 122.50301775396018 240559.4453125
TT_diLept 0.03809437870998642 114101496.0
WJetsToLNu_HT-70To100 380.05058913085185 203239.6796875
WJetsToLNu_HT-100To200 311.53985586101516 268146.140625
WJetsToLNu_HT-200To400 61.5533895601258 395361.265625
WJetsToLNu_HT-400To600 1.8279181987180197 1895963.5625
ZJetsToNuNu_HT-100To200 92.10092407487133 197944.59375
ZJetsToNuNu_HT-200To400 8.996377229024846 610673.78125
ZJetsToNuNu_HT-400To600 3.554660664233398 221

In [ ]:
# with open('histos_bkgs_objects_gbm.dat', 'wb') as histos_file:
#     pickle.dump(histos, histos_file)

In [ ]:
cutFlowInclGrp = {}
for grp in bkg_grps:
    if '60p0' in grp or '5p25' in grp or '52p5' in grp or '6p0' in grp: continue
    for bkg in bkg_grps[grp]:
        if grp in cutFlowInclGrp.keys():
            cutFlowInclGrp[grp] += histos['cutflow_incl'][bkg].astype(int)
        else:
            cutFlowInclGrp[grp] = histos['cutflow_incl'][bkg].astype(int)

pd.DataFrame.from_dict(cutFlowInclGrp)

In [ ]:
cutFlowInclGrp2 = {}
for grp in bkg_grps:
#     if '60p0' in grp or '5p25' in grp or '52p5' in grp or '6p0' in grp: continue
    for bkg in bkg_grps[grp]:
        if grp in cutFlowInclGrp2.keys():
            try:
                cutFlowInclGrp2[grp] += (histos['cutflow_incl'][bkg]*bkgs[bkg]['weight']).astype(int)
            except KeyError: pass
        else:
            try:
                cutFlowInclGrp2[grp] = (histos['cutflow_incl'][bkg]*bkgs[bkg]['weight']).astype(int)
            except KeyError: pass


pd.DataFrame.from_dict(cutFlowInclGrp2)